In [1]:
# Itinializing the Jopyter Notebook file VacationPy.ipynb

In [2]:
# Importing relevant dependencies 
import requests
import json
import pandas as pd
import numpy as np
from config import api_key

# Importing gmaps and Google API key
import gmaps
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [3]:
# Read in cities from WeatherPy output
city_data=pd.read_csv("../WeatherPy/city_data.csv")
city_data.head()

,City name,Longitude,Latitude,"Temperature, F","Humidity, %","Cloudiness, %","Wind speed, mph",Hemisphere
0,abalak,6.328933,15.646237,64.04,69,61,4.30,Northern
1,aban,95.980955,56.847955,51.13,36,100,6.39,Northern
2,abapo,-62.727416,-19.020615,57.29,82,51,1.47,Southern
3,abashiri,144.473037,43.812609,55.78,65,19,2.81,Northern
4,abaza,89.902477,52.131022,66.70,77,99,2.73,Northern


In [12]:
# Set figure dimensions
figure_layout = {
    'width': '100%',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [13]:
# Create a list, which would contain lon, lat coordinates

In [30]:
# Set up gmap with a marker layer containing heatmap of the humidity (%)
locations = city_data[['Latitude', 'Longitude']]
weights = city_data['Humidity, %']
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=200,
                                 point_radius=4))
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…